In [44]:
import time
import os

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains


In [45]:
def getReviews(reviews, dict_):
    for review in reviews:
        review_head = review.find("a", class_="fCitC").get_text()
        review = review.find("q", class_="XllAv H4 _a").get_text()
        dict_[review_head] = review

In [46]:
def getNegativeReviews(hotel_url):
    negativeReviews = {}
    
    driver = webdriver.Firefox()
    driver.get(hotel_url)
    
    driver.find_element_by_xpath(f".//label[contains(text(),'Poor')]").click()
    driver.find_element_by_xpath(f".//label[contains(text(),'Terrible')]").click()

    time.sleep(3)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    reviews = soup.find_all("div", class_="cWwQK MC R2 Gi z Z BB dXjiy")
    
    while True:        
        getReviews(reviews, negativeReviews)        
        
        pagination = soup.find("div", class_="ui_pagination is-centered")
        next_button = pagination.find("a", class_="ui_button nav next primary")
        
        if next_button:
            if "disabled" not in next_button['class']:
                driver.find_element_by_link_text("Next").click()                    
                time.sleep(3)       
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                reviews = soup.find_all("div", class_="cWwQK MC R2 Gi z Z BB dXjiy")
                continue
            else:
                break

        break
    
    driver.close()
    return negativeReviews
    

In [47]:
hotel_url = 'https://www.tripadvisor.com/Hotel_Review-g293890-d305977-Reviews-Hotel_Yak_Yeti-Kathmandu_Kathmandu_Valley_Bagmati_Zone_Central_Region.html'
negative = getNegativeReviews(hotel_url)

In [48]:
negativeDataframe = pd.DataFrame({'heading': negative.keys(),
                                  'reviews': negative.values()})

In [49]:
negativeDataframe

,heading,reviews
0,Not so great experience,I had to stay at the hotel for five days compu...
1,NEVER AGAIN,We had a regional conference for Asia & pacifi...
2,Worst Hospitality,One of the worst five star hospitality till th...
3,The room was a let down it does not feel 5 star,It has a grant entrance and the foyer is amazi...
4,Dilapidated rooms,We were booked at the Yak/Yeti as part of the ...
...,...,...
141,Tri to the mountain Kingdom,We stayed in this hotel Yak and Yeti 19th and ...
142,Fillthy bedspreads and no response whatsoever ...,Stayed here for two nights with a group tourin...
143,A bottle of water is more valuable than the gu...,My experience has been really pathetic. I almo...
144,Severe Mildew Problems,My daughter and I were looking forward to stay...


In [50]:
def concatExcel():
    negativeExcel = pd.read_excel('negative.xlsx', header=0, index_col=0)
    negativeConcat = pd.concat([negativeExcel, negativeDataframe])
    negativeConcat.to_excel('negative.xlsx')

In [51]:
concatExcel()